In [1]:
import requests
import pandas as pd
import json
from dateutil import parser

In [2]:
API_KEY = "e1f7980a69a2fb459301e79beb73433b-257c9fa8b70bd768ec1edb63cae54240"
ACCOUNT_ID = "101-001-25150540-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"


In [3]:
session = requests.Session()

In [4]:
session.headers.update(
    {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
    }
)

In [5]:
params = {
    "count": 10,
    "granularity": "H1",
    "price": "MBA",
}

In [6]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [7]:
response = session.get(url, params=None, data=None, headers=None)

In [8]:
response.status_code

200

In [9]:
data = response.json()

In [10]:
instruments_list = data['instruments']

In [11]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [12]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [13]:
instruments_dict = {}
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {
        k:i[k] for k in key_i
    }

In [31]:
with open("../data/instruments.json",'w') as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [16]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = {
        "count": count,
        "granularity": granularity,
        "price": "MBA",
    }
    response = session.get(url, params=params, data=None, headers=None)
    data= response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data)==0:
        return pd.DataFrame()
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']
    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name,count=10,granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code!=200:
        print("Failed", pair_name)
        return 
    if len(data)==0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} {candles_df.time.min()} {candles_df.time.max()}")
    

In [17]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 9 2023-03-30 13:00:00+00:00 2023-04-02 21:00:00+00:00


In [18]:
our_curr = ['EUR', 'USD','GBP','JPY','CHF','NZD','CAD','AUD']

In [19]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4000 2022-08-10 07:00:00+00:00 2023-04-03 01:00:00+00:00
EUR_USD H4 4000 2020-09-07 09:00:00+00:00 2023-04-02 21:00:00+00:00
EUR_GBP H1 4000 2022-08-10 07:00:00+00:00 2023-04-03 01:00:00+00:00
EUR_GBP H4 4000 2020-09-07 09:00:00+00:00 2023-04-02 21:00:00+00:00
EUR_JPY H1 4000 2022-08-10 07:00:00+00:00 2023-04-03 01:00:00+00:00
EUR_JPY H4 4000 2020-09-07 01:00:00+00:00 2023-04-02 21:00:00+00:00
EUR_CHF H1 4000 2022-08-10 07:00:00+00:00 2023-04-03 01:00:00+00:00
EUR_CHF H4 4000 2020-09-07 09:00:00+00:00 2023-04-02 21:00:00+00:00
EUR_NZD H1 4000 2022-08-10 07:00:00+00:00 2023-04-03 01:00:00+00:00
EUR_NZD H4 4000 2020-09-07 09:00:00+00:00 2023-04-02 21:00:00+00:00
EUR_CAD H1 4000 2022-08-10 07:00:00+00:00 2023-04-03 01:00:00+00:00
EUR_CAD H4 4000 2020-09-07 09:00:00+00:00 2023-04-02 21:00:00+00:00
EUR_AUD H1 4000 2022-08-10 07:00:00+00:00 2023-04-03 01:00:00+00:00
EUR_AUD H4 4000 2020-09-07 09:00:00+00:00 2023-04-02 21:00:00+00:00
USD_JPY H1 4000 2022-08-10 07:00:00+00:00 2023-0